In [1]:
import unittest
import qforte
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermionpsi4 import run_psi4
import numpy

# Do LiH Psi4 calculations and compute direct expectation value

In [2]:
diatomic_bond_length = .7414
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

#Generate molecular data and run psi4 calculation
molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)

molecule = run_psi4(molecule,
                        run_scf=1,
                        run_mp2=0,
                        run_cisd=0,
                        run_ccsd=0,
                        run_fci=0)

print('\nPsi4 calculations:')
print('\nAt bond length of {} angstrom, molecular hydrogen has:'.format(
        diatomic_bond_length))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))

#populate data and integrals
molecule.load()

# Get the Hamiltonian in an active space.
active_space_start = 0
active_space_stop = 4

molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))

# Map operator to fermions and qubits.
fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)
#qubit_hamiltonian.compress()
print('The Jordan-Wigner Molecular Hamiltonian in canonical basis:\n{}'.format(qubit_hamiltonian))

print('\nBuild Qforte Hamiltonian and generator to calculate the expectation value')
qforte_hamiltonian = qforte.build_from_openfermion(qubit_hamiltonian)
#print('\nThe Molecular Hamiltonian in canonical basis:')
#qforte.smart_print(qforte_hamiltonian)

trial_state = qforte.QuantumComputer(8)

#Build a Hartree-Fock trial state
circ = qforte.QuantumCircuit()

# circ.add_gate(qforte.make_gate('X', 0, 0))
# circ.add_gate(qforte.make_gate('X', 1, 1))
# circ.add_gate(qforte.make_gate('X', 2, 2))
# circ.add_gate(qforte.make_gate('X', 3, 3))

# trial_state.apply_circuit(circ)
# qforte.smart_print(trial_state)

exp = trial_state.direct_op_exp_val(qforte_hamiltonian)

print(exp)

#-1.1166843870661929 + 0.0j

#Try with Experiment..

#-7.54543993311268



Psi4 calculations:

At bond length of 0.7414 angstrom, molecular hydrogen has:
Hartree-Fock energy of -7.545439933112644 Hartree.
The Jordan-Wigner Molecular Hamiltonian in canonical basis:
(-4.420874729614635+0j) [] +
(-0.012491446872149356+0j) [X0 X1 Y2 Y3] +
(-0.0028231061946354485+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.0028231061946354485+0j) [X0 X1 X3 X4] +
(-0.003959182752655664+0j) [X0 X1 Y4 Y5] +
(-0.002509949399962252+0j) [X0 X1 Y6 Y7] +
(0.012491446872149356+0j) [X0 Y1 Y2 X3] +
(0.0028231061946354485+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(-0.0028231061946354485+0j) [X0 Y1 Y3 X4] +
(0.003959182752655664+0j) [X0 Y1 Y4 X5] +
(0.002509949399962252+0j) [X0 Y1 Y6 X7] +
(-0.029466845764301027+0j) [X0 Z1 X2] +
(0.00192621079321138+0j) [X0 Z1 X2 X3 Z4 X5] +
(0.00192621079321138+0j) [X0 Z1 X2 Y3 Z4 Y5] +
(0.0037794017446796764+0j) [X0 Z1 X2 Z3] +
(-0.004848141542391399+0j) [X0 Z1 X2 Z4] +
(-0.0044627065486322354+0j) [X0 Z1 X2 Z5] +
(-0.0037738750693342158+0j) [X0 Z1 X2 Z6] +
(-0.0015135563098186847+0j) [X

# Compute Energy via measurement by doing an 'Experiment'

In [3]:
Exper = qforte.Experiment(8, 4 ,circ, qforte_hamiltonian, 100000)

In [4]:
params = []
print(Exper.experimental_avg(params))

-7.545292577119372


In [5]:
Exper2 = qforte.Experiment(8, 4, circ, qforte_hamiltonian, 10000)
params2 = [1.0]
print(Exper2.experimental_avg(params2))

-7.543069700967575
